## Goal: demo training ani on MD trajectories from ASE

In [1]:
import torch

from ase.io import read

from cascade.learning.torchani import TorchANI, estimate_atomic_energies
from cascade.learning.torchani.build import make_aev_computer, make_output_nets

/home/mike/miniconda3/envs/cascade/lib/python3.11/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


In [2]:
traj = read('../0_setup/md/packmol-CH4-in-H2O=32-seed=0-blyp-npt=298/md.traj', 
            index=':')

In [3]:
ref_energies = estimate_atomic_energies(traj)
species = list(ref_energies.keys())
aev = make_aev_computer(species)
nn = make_output_nets(species, aev, 
                      # hidden_units=8, 
                      # hidden_layers=1,
                     )

In [4]:
ani = TorchANI()

In [5]:
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
device

'cuda'

In [6]:
%%time
orig_e, orig_f = ani.evaluate((aev, nn, ref_energies), 
                              traj,
                              #batch_size=,
                              device=device)

CPU times: user 1.67 s, sys: 327 ms, total: 2 s
Wall time: 2.01 s


In [7]:
%%time
model_msg, results = ani.train((aev, nn, ref_energies), 
                         traj[:400], traj[400:], 2)

CPU times: user 5min 9s, sys: 1.62 s, total: 5min 11s
Wall time: 22.5 s


In [8]:
results

,val_force_rmse,val_energy_mae
0,0.479543,0.018704
1,0.441236,0.010166
